<div style="display: flex; background-color: RGB(135,206,235);" >
<h1 style="margin: auto; padding: 30px; ">Application pour détectez des faux billets avec R ou Python</h1>
</div>

In [3]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
import warnings
warnings.simplefilter("ignore", category=UserWarning)

# Charger le modèle Random Forest et le scaler sauvegardés
rf_model = joblib.load('rf_model.pkl')  # Charger le modèle Random Forest
scaler = joblib.load('scaler.pkl')  # Charger le scaler utilisé lors de l'entraînement

# Charger les nouvelles données à tester
df_test0 = pd.read_csv('billets_production.csv', sep=",")  # Remplace par le fichier de test

# Vérifier la présence de la colonne 'id' et la conserver pour la sortie
if "id" not in df_test0.columns:
    raise ValueError("La colonne 'id' est absente du fichier billets_production.csv")

# Prétraitement : Sélectionner les mêmes caractéristiques utilisées lors de l'entraînement
features = ["diagonal", "height_left", "height_right", "margin_low", "margin_up", "length"]
X_new = df_test0[features]  # Caractéristiques à prédire

# Normaliser les nouvelles données avec le même scaler utilisé pour l'entraînement
X_new_scaled = scaler.transform(X_new)  # Appliquer la même transformation

# Appliquer le modèle Random Forest pour prédire les classes et les probabilités
y_new_pred = rf_model.predict(X_new_scaled)  # Prédictions des classes
y_new_proba = rf_model.predict_proba(X_new_scaled)[:, 1]  # Probabilité d'être un vrai billet (classe 1)

# Ajouter les prédictions et probabilités dans le DataFrame
df_test0["Predicted Class"] = y_new_pred
df_test0["Proba_Vrai"] = y_new_proba  # Ajout des probabilités

# Sauvegarder les résultats dans un fichier CSV
df_test0.to_csv('test_avec_predictions_rf.csv', index=False)

# Afficher les premières lignes du fichier avec les prédictions et probabilités
df_test0.head(10)


diagonal  height_left  height_right  margin_low  margin_up  length   id  \
0    171.76       104.01        103.54        5.21       3.30  111.42  A_1   
1    171.87       104.17        104.13        6.00       3.31  112.09  A_2   
2    172.00       104.58        104.29        4.99       3.39  111.57  A_3   
3    172.49       104.55        104.34        4.44       3.03  113.20  A_4   
4    171.65       103.63        103.56        3.77       3.16  113.33  A_5   

   Predicted Class  Proba_Vrai  
0            False        0.00  
1            False        0.00  
2            False        0.00  
3             True        0.98  
4             True        1.00